<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#Renaming" data-toc-modified-id="Renaming-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Renaming</a></span><ul class="toc-item"><li><span><a href="#Writing-output" data-toc-modified-id="Writing-output-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Writing output</a></span></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Rename the tips of the GTDBr89 Struo DB phylogeny to match the kraken/bracken output (species names)

# Var

In [1]:
# GTDBr89 phylogeny
phy_file = '/ebio/abt3_projects/databases_no-backup/GTDB/release89/LLMGP-DB/phylogeny/ar122-bac120_r89_1per-GTDB-Spec_gte50comp-lt5cont.nwk'
# GTDBr89 metadata
meta_file = '/ebio/abt3_projects/databases_no-backup/GTDB/release89/LLMGP-DB/metadata_1per-GTDB-Spec_gte50comp-lt5cont_wtaxID_wPath.tsv'
# params
threads = 8

# Init

In [4]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(tidytable)
library(ape)
library(LeyLabRMisc)

In [5]:
df.dims()
setDTthreads(threads)

# Load

In [6]:
phy = read.tree(phy_file)
phy


Phylogenetic tree with 23360 tips and 23359 internal nodes.

Tip labels:
  RS_GCF_000296615.1, RS_GCF_000337335.1, RS_GCF_002355655.1, RS_GCF_001280455.1, RS_GCF_000337035.1, RS_GCF_002114285.1, ...
Node labels:
  100.0, d__Archaea100.0, 97.0, 99.0, 100.0, 100.0:p__Halobacterota, ...

Rooted; includes branch lengths.

In [7]:
# reading metadata
tax_levs = c('Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species')
meta = Fread(meta_file) %>%
    select.(accession, ncbi_organism_name, gtdb_taxonomy) %>%
    separate.(gtdb_taxonomy, tax_levs, sep=';') 
meta %>% unique_n('accessions', accession)
meta

No. of unique accessions: 23360 


accession,ncbi_organism_name,Domain,Phylum,Class,Order,Family,Genus,Species
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GB_GCA_002789635.1,GB_GCA_002789635.1_Candidatus Aenigmarchaeota archaeon CG_4_9_14_3_um_filter_37_18,d__Archaea,p__Aenigmarchaeota,c__Aenigmarchaeia,o__CG10238-14,f__CG10238-14,g__CG10238-14,s__CG10238-14 sp002789635
GB_GCA_001784635.1,GB_GCA_001784635.1_Candidatus Micrarchaeota archaeon RBG_16_49_10,d__Archaea,p__Aenigmarchaeota,c__Aenigmarchaeia,o__CG10238-14,f__CG10238-14,g__RBG-16-49-10,s__RBG-16-49-10 sp001784635
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
GB_GCA_001775395.1,GB_GCA_001775395.1_candidate division Zixibacteria bacterium RBG_16_53_22,d__Bacteria,p__Zixibacteria,c__MSB-5A5,o__UBA10806,f__UBA10806,g__UBA10806,s__UBA10806 sp001775395
GB_GCA_002791595.1,GB_GCA_002791595.1_candidate division Zixibacteria bacterium CG_4_9_14_3_um_filter_46_8,d__Bacteria,p__Zixibacteria,c__MSB-5A5,o__UM-FILTER-46-8,f__UM-FILTER-46-8,g__UM-FILTER-46-8,s__UM-FILTER-46-8 sp002791595


# Renaming

In [8]:
# checking overlap
overlap(phy$tip.label, meta, sel_col_y=accession)

intersect(x,y): 23360 
setdiff(x,y): 0 
setdiff(y,x): 0 
union(x,y): 23360 


In [9]:
# renaming tips
tmp = meta %>%
    as.data.frame
rownames(tmp) = tmp$accession

phy$tip.label = tmp[phy$tip.label,]$Species
rm(tmp)
gc()

phy

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,958093,51.2,2600157,138.9,1715397,91.7
Vcells,3222385,24.6,10247656,78.2,10236877,78.2



Phylogenetic tree with 23360 tips and 23359 internal nodes.

Tip labels:
  s__Halorubrum sp000296615, s__Halorubrum distributum, s__Halorubrum trapanicum, s__Halorubrum tropicale, s__Halorubrum coriense, s__Halorubrum ezzemoulense, ...
Node labels:
  100.0, d__Archaea100.0, 97.0, 99.0, 100.0, 100.0:p__Halobacterota, ...

Rooted; includes branch lengths.

## Writing output

In [10]:
outF = file.path(dirname(phy_file), 'ar122-bac120_r89_1per-GTDB-Spec_gte50comp-lt5cont_rn.nwk')
write.tree(phy, outF)
cat('File written:', outF, '\n')

File written: /ebio/abt3_projects/databases_no-backup/GTDB/release89/LLMGP-DB/phylogeny/ar122-bac120_r89_1per-GTDB-Spec_gte50comp-lt5cont_rn.nwk 


# sessionInfo

In [11]:
sessionInfo()

R version 3.6.2 (2019-12-12)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.3 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/phyloseq/lib/libopenblasp-r0.3.7.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] LeyLabRMisc_0.1.3 ape_5.3           tidytable_0.3.2   data.table_1.12.8
[5] ggplot2_3.2.1     tidyr_1.0.0       dplyr_0.8.3      

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.3       pillar_1.4.3     compiler_3.6.2   base64enc_0.1-3 
 [5] tools_3.6.2      zeallot_0.